In [ ]:
# !pip install langchain-core langchain-openai langchain-community pydantic

## 1. 환경 설정 및 모델 준비

### 기본 라이브러리 불러오기

In [ ]:
import os
import getpass

###  OpenAI 키 세팅

In [ ]:
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except (ImportError, KeyError):
    try:
        api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
        os.environ["OPENAI_API_KEY"] = api_key
        print("API 키가 입력되었습니다.")
    except Exception as e:
        print(f"API 키를 설정하는 중 오류가 발생했습니다: {e}")
        exit()

## 2. LLM 및 기본 템플릿

In [ ]:
from langchain_openai import ChatOpenAI

### OpenAI LLM 호출

In [ ]:
llm_openai = ChatOpenAI(model="gpt-5-nano")
response1 = llm_openai.invoke("태양계에서 가장 큰 행성은?")
print(response1.content)

### 파라미터를 적용한 OpenAI LLM 호출

In [ ]:
llm_openai_params = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
)

response3 = llm_openai_params.invoke("태양계에서 두 번째로 큰 행성은?")
print(response3.content)

### 프롬프트 템플릿 적용

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# 메시지 리스트를 사용한 호출
messages = [
    SystemMessage(content="당신은 우주에 대해 모든 것을 알고 있는 천문학자입니다."),
    HumanMessage(content="우리 은하에서 가장 가까운 은하는?"),
]
response4 = llm_openai.invoke(messages)
print(response4.content)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# MessagesPlaceholder는 e. 메모리 파트에서 다룰 것이므로 여기서는 제외합니다.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 {country} 요리 전문 셰프입니다. 해당 분야에 대해서만 자세하게 답변하고, 다른 나라 음식에 대해 질문할 경우 {country} 방식으로 재해석한 레시피를 제안하세요."),
    ("human", "{question}"),
])

formatted_prompt = prompt_template.invoke({
    "country": "한국의 전통 남도식",
    "question": "까르보나라를 만드는 가장 전통적인 방법은 무엇인가요?",
})
print(formatted_prompt.to_messages())

In [ ]:
llm_openai = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.0,
                        max_tokens=1000,
                        )

llm_openai.invoke(formatted_prompt) # 또는 llm_openai.invoke(formatted_prompt.to_messages())

### String output parser 적용

In [ ]:
# StrOutputParser의 기능
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

llm_output = llm_openai.invoke(formatted_prompt.to_messages())
parsed_output = output_parser.invoke(llm_output)
print(parsed_output)

## 3. LCEL(LangXhain Expression Language)와 체인

- LangChain의 구성요소(Prompt, LLM, Parser 등)는 'Runnable' 프로토콜을 따릅니다.
- 이는 각 요소가 `.invoke()`, `.batch()`, `.stream()` 메소드를 가지고 있음을 의미하며,
- 이 덕분에 파이프(`|`) 연산자로 쉽게 연결하여 '체인'을 만들 수 있습니다.

In [ ]:
# LCEL을 이용한 간단한 체인 구성

chain = prompt_template | llm_openai | output_parser

# .invoke(): 하나의 입력을 받아 결과를 반환합니다.
response_from_chain = chain.invoke({
    "country": "한국의 전통 남도식",
    "question": "시카고 피자 만드는 방법 알려주세요.",
})
print(response_from_chain)

In [ ]:
# .batch(): 여러 입력을 리스트로 받아 결과 리스트를 반환합니다. (병렬 처리로 더 빠름)
batch_inputs = [
    {"country": "일본", "question": "스시의 역사에 대해 알려줘."},
    {"country": "프랑스", "question": "크루아상의 기원은 어디야?"},
]
batch_responses = chain.batch(batch_inputs)
for res in batch_responses:
    print(f"- {res[:50]}...")

In [ ]:
# .stream(): 결과가 생성되는 대로 실시간으로 조각(chunk)을 받아 처리합니다.
stream = chain.stream({
    "country": "인도",
    "question": "치킨 티카 마살라 레시피 알려줘.",
})
for chunk in stream:
    print(chunk, end="", flush=True)
print() # 마지막에 줄바꿈을 위해 추가

## 4. 메모리

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

# 메모리 테스트를 위한 간단한 체인 구성
prompt_for_memory = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자와 대화하는 친절한 AI입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

chat_chain = prompt_for_memory | llm_openai | output_parser

In [ ]:
# history를 직접 빈 리스트로 전달할 경우 대화가 이어지지 않음
print("User: 제 이름은 오근철입니다.")
response = chat_chain.invoke({"input": "제 이름은 오근철입니다.", "history": []})
print(f"AI: {response}")

print("\nUser: 제 이름이 뭐라고 했죠?")
response = chat_chain.invoke({"input": "제 이름이 뭐라고 했죠?", "history": []})
print(f"AI: {response}") # 이름을 기억하지 못합니다.

In [ ]:
# 메모리가 추가된 체인
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    lambda session_id: memory.chat_memory,
    input_messages_key="input",
    history_messages_key="history",
)
config = {"configurable": {"session_id": "user_session_1"}}

print("User: 제 이름은 오근철입니다.")
response = chain_with_memory.invoke({"input": "제 이름은 오근철입니다."}, config=config)
print(f"AI: {response}")

print("\nUser: 제 이름이 뭐라고 했죠?")
response = chain_with_memory.invoke({"input": "제 이름이 뭐라고 했죠?"}, config=config)
print(f"AI: {response}") # 이름을 정확히 기억하고 대답합니다.

In [ ]:
#  Assistant 메시지를 이용한 수동 메모리 관리

# 대화 기록을 수동으로 관리할 리스트
manual_history = []

# 첫 번째 질문
print("\nUser: 제 취미는 독서입니다. 영국 소설을 좋아해요.")
user_input_1 = "제 취미는 독서입니다. 영국 소설을 좋아해요."
manual_history.append(HumanMessage(content=user_input_1))
response_manual_1 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_1, "history": manual_history}))

# AI의 답변(AIMessage)을 기록에 추가
manual_history.append(response_manual_1)
print(f"AI: {response_manual_1.content}")

# 두 번째 질문
print("\nUser: 특히, 그 중에서 추리소설을 좋아해요.")
user_input_2 = "특히, 그 중에서 추리소설을 좋아해요."
manual_history.append(HumanMessage(content=user_input_2))

# 이전 대화 기록이 담긴 history를 함께 전달
response_manual_2 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_2, "history": manual_history}))
print(f"AI: {response_manual_2.content}") # 취미를 정확히 기억하고 대답합니다.

print("\nUser: 제 취향을 고려해서 재미있는 책들을 추천해주세요.")
user_input_3 = "제 취향을 고려해서 재미있는 책들을 추천해주세요."
manual_history.append(HumanMessage(content=user_input_3))

response_manual_3 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_3, "history": manual_history}))
print(f"AI: {response_manual_3.content}")

## 5. 출력 구조화를 위한 파서

### JSON Output Parser

In [ ]:
# JsonOutputParser
from langchain_core.output_parsers import JsonOutputParser
json_prompt = ChatPromptTemplate.from_template(
    """문장에서 '인물', '장소', '시간' 정보를 추출하여 JSON으로 출력해줘.
    문장: {sentence}
    JSON 출력:"""
)
json_chain = json_prompt | llm_openai | JsonOutputParser()
json_output = json_chain.invoke({"sentence": "내일 오후 3시에 서울역에서 김철수 씨를 만나기로 했어요."})
print(json_output)

### Pydantic Output Parser

In [ ]:
# PydanticOutputParser
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# 파서 클래스 정의
class MeetingDetails(BaseModel):
    location: Optional[str] = Field(description="회의가 열리는 장소")
    time: Optional[str] = Field(description="회의 시작 시간")
    attendees: List[str] = Field(description="회의 참석자들의 이름 목록", default=[])

# 파서 생성
pydantic_parser = PydanticOutputParser(pydantic_object=MeetingDetails)

# 프롬프트 템플릿 생성
pydantic_prompt = ChatPromptTemplate.from_template(
    """주어진 이메일 본문에서 회의 정보를 추출해줘.
    {format_instructions}
    이메일 본문: --- {email_body} ---
    """
    )

# 파서가 적용된 체인 생성
pydantic_chain = pydantic_prompt | llm_openai | pydantic_parser

In [ ]:
sample_email = """
안녕하세요,
다음 주 프로젝트 리뷰 회의 일정을 공유드립니다.
- 일시: 2025년 8월 20일 (수) 오후 2시
- 장소: 본사 3층 대회의실
- 참석자: 김태균 팀장, 안가영 선임, 조예찬 책임
감사합니다.
"""
meeting_info = pydantic_chain.invoke({
    "email_body": sample_email,
    "format_instructions": pydantic_parser.get_format_instructions()
})
print(meeting_info)